<a href="https://colab.research.google.com/github/Pelosomelo/Misp/blob/main/instalacja_pierwsze_cwiczenia_z_Pulp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp


Reading package lists... Done
Building dependency tree       
Reading state information... Done
coinor-cbc is already the newest version (2.9.9+repack1-1).
coinor-clp is already the newest version (1.16.11+repack1-1).
glpk-utils is already the newest version (4.65-1).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [ ]:
### kod sprawdza czy instalacja się udała
import pulp

print(pulp.__version__)

2.3.1


In [ ]:
pulp.pulpTestAll()

# Zadania:

1. przeczytaj materiały dotyczące pulp ze strony internetowej https://coin-or.github.io/pulp/main/index.html oraz przejdź przez blending problem (przepisz do swojego notebooka)

2. w blending problem dodaj zmienną decyzyjną. Stwórz zmienną która pojawi się w końcowym składzie produktu (tj. wartość zmiennej będzie miała niezerową postać)

3. Dodaj ograniczenie co do zawartości witaminy B 

Zawartość:

| Nazwa      | zawartość ug/100g |
|------------|-------------------|
| chicken    | 0.3               |
| beef       | 2.6               |
| mutton     | 2.6               |
| rice       | 0                 |
| wheat bran | 0                 |
| gel        | 0                 |
| sardines   | 8.9               |

Kot ma zjeść minimum 0.24 ug na kilogram masy więc około
0.8 ug dziennie

4. Na końcu ma pojawiać się zdanie podsumowujące wynik, w stylu:
Końcowy produkt będzie kosztować {}, będzie miał w składzie {nazwa}: {} %, {nastepna nazwa}: {} %

Takie zdanie można skleić z części generowanych automatycznie:



In [ ]:
sklejone_produkt_cena = [":".join((produkt, str(cena))) for produkt,cena in zip(["jabłko","banan"],[5,10])]
print(sklejone_produkt_cena)
sklejona_calosc = ", ".join(sklejone_produkt_cena)

print(sklejona_calosc)

['jabłko:5', 'banan:10']
jabłko:5, banan:10


In [ ]:
from pulp import *
prob = LpProblem("The Whiskas Problem",LpMinimize)
LpVariable("example", None, 100)
x1=LpVariable("ChickenPercent",0,None,LpInteger)
x2=LpVariable("BeefPercent",0)
x3=LpVariable("KanapkaPercent",0)
prob += 0.013*x1 + 0.008*x2 + 0.012*x3, "Total Cost of Ingredients per can"
prob += x1 + x2 + x3 == 100, "PercentagesSum"
prob += 0.100*x1 + 0.200*x2 +0.050*x3 >= 8.0, "ProteinRequirement"
prob += 0.080*x1 + 0.100*x2 +0.100*x3>= 6.0, "FatRequirement"
prob += 0.001*x1 + 0.005*x2 +0.0005*x3<= 2.0, "FibreRequirement"
prob += 0.002*x1 + 0.005*x2 +0.004*x3<= 0.4, "SaltRequirement"
prob += 0.300*x1 + 2.600*x2 +0.004*x3>= 0.8, "Witamina B"
prob.writeLP("WhiskasModel.lp")
prob.solve()
print("Status:", LpStatus[prob.status])
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Total Cost of Ingredients per can = ", value(prob.objective))
print("Całkowity koszt = ", value(prob.objective),", będzie miał w składzie:","ChickenPercent:",x1.varValue,"% ,BeefPercent:",x2.varValue,"% ,KanapkaPercent:",x3.varValue,"%")

ModuleNotFoundError: ignored

In [ ]:
from pulp import *
Ingredients = ['CHICKEN', 'BEEF', 'MUTTON', 'RICE', 'WHEAT', 'GEL','Kanapka']

# A dictionary of the costs of each of the Ingredients is created
costs = {'CHICKEN': 0.013, 
         'BEEF': 0.008, 
         'MUTTON': 0.010, 
         'RICE': 0.002, 
         'WHEAT': 0.005, 
         'GEL': 0.001,
         'Kanapka':0.012}

# A dictionary of the protein percent in each of the Ingredients is created
proteinPercent = {'CHICKEN': 0.100, 
                  'BEEF': 0.200, 
                  'MUTTON': 0.150, 
                  'RICE': 0.000, 
                  'WHEAT': 0.040, 
                  'GEL': 0.000,
                  'Kanapka':0.050}

# A dictionary of the fat percent in each of the Ingredients is created
fatPercent = {'CHICKEN': 0.080, 
              'BEEF': 0.100, 
              'MUTTON': 0.110, 
              'RICE': 0.010, 
              'WHEAT': 0.010, 
              'GEL': 0.000,
              'Kanapka':0.200}

# A dictionary of the fibre percent in each of the Ingredients is created
fibrePercent = {'CHICKEN': 0.001, 
                'BEEF': 0.005, 
                'MUTTON': 0.003, 
                'RICE': 0.100, 
                'WHEAT': 0.150, 
                'GEL': 0.000,
                'Kanapka':0.0005}

# A dictionary of the salt percent in each of the Ingredients is created
saltPercent = {'CHICKEN': 0.002, 
               'BEEF': 0.005, 
               'MUTTON': 0.007, 
               'RICE': 0.002, 
               'WHEAT': 0.008, 
               'GEL': 0.000,
               'Kanapka':0.004}
# A dictionary of the Kanapka percent in each of the Ingredients is created
WitaminaBPercent = {'CHICKEN': 0.3, 
               'BEEF': 2.6, 
               'MUTTON': 2.6, 
               'RICE': 0.0, 
               'WHEAT': 0.00, 
               'GEL': 0.000,
               'Kanapka':0.004}
prob = LpProblem("The Whiskas Problem", LpMinimize)
ingredient_vars = LpVariable.dicts("Ingr",Ingredients,0)
prob += lpSum([costs[i]*ingredient_vars[i] for i in Ingredients]),"Total Cost of Ingredients per can"
prob += lpSum([ingredient_vars[i] for i in Ingredients]) == 100, "PercentagesSum"
prob += lpSum([proteinPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 8.0, "ProteinRequirement"
prob += lpSum([fatPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 6.0, "FatRequirement"
prob += lpSum([fibrePercent[i] * ingredient_vars[i] for i in Ingredients]) <= 2.0, "FibreRequirement"
prob += lpSum([saltPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4, "SaltRequirement"
prob += lpSum([WitaminaBPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 0.8, "WitaminaBRequirement"

prob.solve()
print("Całkowity koszt = ", value(prob.objective),", będzie miał w składzie:")
for v in prob.variables():
    print(v.name, "=", v.varValue)




Całkowity koszt =  0.48571427899999997 , będzie miał w składzie:
Ingr_BEEF = 37.142857
Ingr_CHICKEN = 0.0
Ingr_GEL = 51.428571
Ingr_Kanapka = 11.428571
Ingr_MUTTON = 0.0
Ingr_RICE = 0.0
Ingr_WHEAT = 0.0


/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
